In [1]:
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pickle
import joblib
import pandas as pd
import psycopg2
import os
import csv

folder = "D:/TEST/airflow/project/docker-airflow/data/"

iris = datasets.load_iris()
clf = SVC(gamma=0.001, C=100.)
clf.fit(iris.data[:-1], iris.target[:-1])
y_predict = clf.predict(iris.data)
accuracy_score(y_predict, iris.target)

with open(f'{folder}clf.pickle', 'wb') as f:
     pickle.dump(clf, f)
        
with open(f'{folder}clf.pickle', 'rb') as f:
     clf_new = pickle.load(f)
        
import joblib
filename = f'{folder}clf.pkl'
_ = joblib.dump(clf, filename, compress=9)

clf_new = joblib.load(filename)
y_predict_new = clf_new.predict(iris.data)
accuracy_score(y_predict_new, iris.target)

type(clf)
type(clf_new)

y_predict_new = clf_new.predict(iris.data)
accuracy_score(y_predict_new, iris.target)

0.98

In [2]:
df_iris = pd.read_csv(f'{folder}iris.csv')
columns_name = list(map(lambda x: x.replace('.','_'), list(df_iris.columns[:-1])))
df_iris_data = df_iris[df_iris.columns[:-1]]
df_iris_data.columns = columns_name
df_iris_data.to_csv(f'{folder}iris_data.csv',index=False)
# df_iris_data = pd.read_csv('iris_data.csv')

os.environ["HOST_DB"] = '127.0.0.1'
os.environ["HOST_DB_port"] = '5423'
os.environ["DB_name"] = 'postgres'
os.environ["USER_DB"] ='postgres'
os.environ["PASSWORD_DB"] = 'postgres'

conn = psycopg2.connect(host=os.environ["HOST_DB"], port = os.environ["HOST_DB_port"], database=os.environ["DB_name"], user=os.environ["USER_DB"], password=os.environ["PASSWORD_DB"])
cur = conn.cursor()

column_0 = df_iris.columns[0].replace('.','_')
column_1 = df_iris.columns[1].replace('.','_')
column_2 = df_iris.columns[2].replace('.','_')
column_3 = df_iris.columns[3].replace('.','_')
column_4 = "is_worked"

cur.execute(f"CREATE TABLE IF NOT EXISTS table_base (id serial PRIMARY KEY, {column_0} float, {column_1} float,{column_2} float,{column_3} float,{column_4} int);")

reader = csv.reader(open(f'{folder}iris_data.csv', 'r'))

for i, row in enumerate(reader):
    print(i, row)
    if i == 0: continue
    cur.execute('''
        INSERT INTO "table_base"(
            "sepal_length", "sepal_width", "petal_length", "petal_width", "is_worked"
        ) values (%s,%s,%s,%s,0)''', row
    )
    
cur.execute(f"CREATE TABLE IF NOT EXISTS table_transform (id serial PRIMARY KEY, {column_0} float, {column_1} float,{column_2} float,{column_3} float,{column_4} int);")

cur.execute(f"CREATE TABLE IF NOT EXISTS table_predict (id serial PRIMARY KEY, iris_type smallint);")

conn.commit()
conn.close()
cur.close()

0 ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
1 ['5.1', '3.5', '1.4', '0.2']
2 ['4.9', '3.0', '1.4', '0.2']
3 ['4.7', '3.2', '1.3', '0.2']
4 ['4.6', '3.1', '1.5', '0.2']
5 ['5.0', '3.6', '1.4', '0.2']
6 ['5.4', '3.9', '1.7', '0.4']
7 ['4.6', '3.4', '1.4', '0.3']
8 ['5.0', '3.4', '1.5', '0.2']
9 ['4.4', '2.9', '1.4', '0.2']
10 ['4.9', '3.1', '1.5', '0.1']
11 ['5.4', '3.7', '1.5', '0.2']
12 ['4.8', '3.4', '1.6', '0.2']
13 ['4.8', '3.0', '1.4', '0.1']
14 ['4.3', '3.0', '1.1', '0.1']
15 ['5.8', '4.0', '1.2', '0.2']
16 ['5.7', '4.4', '1.5', '0.4']
17 ['5.4', '3.9', '1.3', '0.4']
18 ['5.1', '3.5', '1.4', '0.3']
19 ['5.7', '3.8', '1.7', '0.3']
20 ['5.1', '3.8', '1.5', '0.3']
21 ['5.4', '3.4', '1.7', '0.2']
22 ['5.1', '3.7', '1.5', '0.4']
23 ['4.6', '3.6', '1.0', '0.2']
24 ['5.1', '3.3', '1.7', '0.5']
25 ['4.8', '3.4', '1.9', '0.2']
26 ['5.0', '3.0', '1.6', '0.2']
27 ['5.0', '3.4', '1.6', '0.4']
28 ['5.2', '3.5', '1.5', '0.2']
29 ['5.2', '3.4', '1.4', '0.2']
30 ['4.7', '3.2'